In [2]:
import pypdb

res_name = 'DMS'
chem_desc = pypdb.describe_chemical(f"{res_name}")

In [3]:
new_smiles = chem_desc["describeHet"]["ligandInfo"]["ligand"]["smiles"]
print(new_smiles)

CS(=O)C


In [4]:
import pandas as pd
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
import pypdb

xc_search = pd.read_csv('xcsearch.csv')
xcdb_f = open('smiles_list.txt', 'r').read()
xcdb_smiles = xcdb_f.split(',')
xcdb_mols = [Chem.MolFromSmiles(m) for m in xcdb_smiles]
xcdb_fps = []
c = 0
for mol in xcdb_mols:
    try:
        fp = AllChem.GetMorganFingerprint(mol,2)
        xcdb_fps.append(fp)
    except:
        c += 1

RDKit ERROR: [22:05:43] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [22:05:43] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [22:05:43] Explicit valence for atom # 8 N, 4, is greater than permitted
RDKit ERROR: [22:05:43] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [22:05:43] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [22:05:43] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [22:05:43] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [22:05:44] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [22:05:44] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [22:05:44] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [22:05:44] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [22:05:44] Explicit valence for atom #

In [5]:
non_ligs = eval(open('non_ligs.json', 'r').read())

pdb_matches = []

for _,row in xc_search.iterrows():
    if row['compound_id']:
        try:
            cmpd_ids = row['compound_id'].split(',')
        except:
            continue
    for cid in cmpd_ids:
        if cid in non_ligs: 
            continue
        try:
            chem_desc = pypdb.describe_chemical(f"{cid}")
            new_smiles = chem_desc["describeHet"]["ligandInfo"]["ligand"]["smiles"]
            can_mol = Chem.MolFromSmiles(new_smiles)
            can_no_h = Chem.rdmolops.RemoveHs(can_mol, sanitize=True)
            can_smi = Chem.MolToSmiles(can_no_h, isomericSmiles=False)
            can_can_mol = Chem.MolFromSmiles(can_smi)
            can_fp = AllChem.GetMorganFingerprint(can_can_mol,2)
            for comparison in xcdb_fps:
                sim = DataStructs.DiceSimilarity(can_fp,comparison)
                if sim==1.0:
                    print(can_smi)
                    pdb_matches.append(row['pdb_id'])
                    print(pdb_matches)
                    continue
        #             if can_smi in xcdb_smiles:
        #                 print(can_smi)
        except:
            continue

/opt/miniconda3/envs/fragalysis_env2/lib/python3.7/site-packages/pypdb/pypdb.py:414: UserWarning: Retrieval failed, returning None
  warnings.warn("Retrieval failed, returning None")


O=C(O)C(O)C(O)C(=O)O
['4uov']


RDKit ERROR: [22:06:37] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [22:06:40] Explicit valence for atom # 0 O, 3, is greater than permitted


Nc1ccc(CC(N)C(=O)O)cc1
['4uov', '5bt0']
NCC(=O)O
['4uov', '5bt0', '5a3a']
CCC(=O)O
['4uov', '5bt0', '5a3a', '5a7r']
COC(=O)c1cccc(NS(C)(=O)=O)c1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6']
C=CCN
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1']
Cc1n[nH]c(=O)nc1Nc1ccc(C(C)C)cc1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9']
c1ccc2[nH]nnc2c1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz']
COc1ccc(NCc2ccccn2)cc1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch']
Nc1cccc(C(=O)O)c1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv']
N#Cc1ccccc1O
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1']
Nc1ncnc2[nH]cnc12
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o']
Oc1ccc(O)c2c1C1CCC2C1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3']
O=C(O)CC(O)C(=O)O
['4uov', '5bt0', '5a3a', '5a7r

N#Cc1ccc(O)cc1F
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo']
Cn1cccc(O)c1=S
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo', '5qhm']
Cc1cccc(Nc2ncnc3c2cnn3C)c1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5n

...

Cc1cc(C)nc(Nc2ccccc2)n1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo', '5qhm', '5qjl', '5qpn', '5qrf', '5rh3', '5ryr', '5qet', '5qgm', '5qie', '5qjx', '5qqw', '5qsa', '5r5k', '5r98', '5rf3', '5rgg', '5rlu', '5ryq', '5o1o', '5qdl', '5qi2', '5poz', '5qey', '5qg5', '5qhx', '5qjv', '5qf9', '5qgx', '5qry', '5ofw', '5qfc', '5qjb', '5qq4', '5r64', '5r9w', '5nzp', '5qoz', '5qtn', '5r5x', '5rhq', '5ry2', '5rzy', '5r51', '5rhi', '5rlj', '5rzw', '5os3', '5os2', '5q1s', '5qe9', '5qg9', '5qi3', '5qk0', '5qrt', '5r59', '5rho', '5rw7', '5qs4', '5qxk', '5r4n', '5r83', '5rgi', '5rjv', '5rll', '5rx7', '5rz1', '5qdu', '5qfh', '5qgw', '5qqk', '5qqk', '5r4j', '5rhg', '5rl9', '5rwi', '5ryd', '5q1r', '5qdi',

Cn1c(=O)c2[nH]cnc2n(C)c1=O
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo', '5qhm', '5qjl', '5qpn', '5qrf', '5rh3', '5ryr', '5qet', '5qgm', '5qie', '5qjx', '5qqw', '5qsa', '5r5k', '5r98', '5rf3', '5rgg', '5rlu', '5ryq', '5o1o', '5qdl', '5qi2', '5poz', '5qey', '5qg5', '5qhx', '5qjv', '5qf9', '5qgx', '5qry', '5ofw', '5qfc', '5qjb', '5qq4', '5r64', '5r9w', '5nzp', '5qoz', '5qtn', '5r5x', '5rhq', '5ry2', '5rzy', '5r51', '5rhi', '5rlj', '5rzw', '5os3', '5os2', '5q1s', '5qe9', '5qg9', '5qi3', '5qk0', '5qrt', '5r59', '5rho', '5rw7', '5qs4', '5qxk', '5r4n', '5r83', '5rgi', '5rjv', '5rll', '5rx7', '5rz1', '5qdu', '5qfh', '5qgw', '5qqk', '5qqk', '5r4j', '5rhg', '5rl9', '5rwi', '5ryd', '5q1r', '5qd

OCc1cn(-c2ccc(Cl)cc2)nn1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo', '5qhm', '5qjl', '5qpn', '5qrf', '5rh3', '5ryr', '5qet', '5qgm', '5qie', '5qjx', '5qqw', '5qsa', '5r5k', '5r98', '5rf3', '5rgg', '5rlu', '5ryq', '5o1o', '5qdl', '5qi2', '5poz', '5qey', '5qg5', '5qhx', '5qjv', '5qf9', '5qgx', '5qry', '5ofw', '5qfc', '5qjb', '5qq4', '5r64', '5r9w', '5nzp', '5qoz', '5qtn', '5r5x', '5rhq', '5ry2', '5rzy', '5r51', '5rhi', '5rlj', '5rzw', '5os3', '5os2', '5q1s', '5qe9', '5qg9', '5qi3', '5qk0', '5qrt', '5r59', '5rho', '5rw7', '5qs4', '5qxk', '5r4n', '5r83', '5rgi', '5rjv', '5rll', '5rx7', '5rz1', '5qdu', '5qfh', '5qgw', '5qqk', '5qqk', '5r4j', '5rhg', '5rl9', '5rwi', '5ryd', '5q1r', '5qdi'

O=C(OCc1ccc(Cl)cc1)c1cccnc1
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo', '5qhm', '5qjl', '5qpn', '5qrf', '5rh3', '5ryr', '5qet', '5qgm', '5qie', '5qjx', '5qqw', '5qsa', '5r5k', '5r98', '5rf3', '5rgg', '5rlu', '5ryq', '5o1o', '5qdl', '5qi2', '5poz', '5qey', '5qg5', '5qhx', '5qjv', '5qf9', '5qgx', '5qry', '5ofw', '5qfc', '5qjb', '5qq4', '5r64', '5r9w', '5nzp', '5qoz', '5qtn', '5r5x', '5rhq', '5ry2', '5rzy', '5r51', '5rhi', '5rlj', '5rzw', '5os3', '5os2', '5q1s', '5qe9', '5qg9', '5qi3', '5qk0', '5qrt', '5r59', '5rho', '5rw7', '5qs4', '5qxk', '5r4n', '5r83', '5rgi', '5rjv', '5rll', '5rx7', '5rz1', '5qdu', '5qfh', '5qgw', '5qqk', '5qqk', '5r4j', '5rhg', '5rl9', '5rwi', '5ryd', '5q1r', '5q

[H]N=C(NCC)NCC1C2CC3CC(C2)CC1C3
['4uov', '5bt0', '5a3a', '5a7r', '5kh6', '5jp1', '5kh9', '5llz', '5kch', '5csv', '5fy1', '5f7o', '5fz3', '5fy0', '5fz7', '5enb', '5fz9', '5cqa', '5fz1', '5aoa', '5b8d', '5g2u', '5cub', '5enc', '5fzb', '5fzh', '5jp3', '5oui', '5nzq', '5q22', '5qdr', '5qfb', '5qgs', '5qjf', '5qpk', '5qrc', '5qxj', '5r54', '5rfd', '5rkp', '5ry4', '5rzb', '5qe6', '5qfo', '5qhm', '5qjl', '5qpn', '5qrf', '5rh3', '5ryr', '5qet', '5qgm', '5qie', '5qjx', '5qqw', '5qsa', '5r5k', '5r98', '5rf3', '5rgg', '5rlu', '5ryq', '5o1o', '5qdl', '5qi2', '5poz', '5qey', '5qg5', '5qhx', '5qjv', '5qf9', '5qgx', '5qry', '5ofw', '5qfc', '5qjb', '5qq4', '5r64', '5r9w', '5nzp', '5qoz', '5qtn', '5r5x', '5rhq', '5ry2', '5rzy', '5r51', '5rhi', '5rlj', '5rzw', '5os3', '5os2', '5q1s', '5qe9', '5qg9', '5qi3', '5qk0', '5qrt', '5r59', '5rho', '5rw7', '5qs4', '5qxk', '5r4n', '5r83', '5rgi', '5rjv', '5rll', '5rx7', '5rz1', '5qdu', '5qfh', '5qgw', '5qqk', '5qqk', '5r4j', '5rhg', '5rl9', '5rwi', '5ryd', '5q1r',

In [6]:
unique_matches = set(pdb_matches)
print(len(unique_matches))
with open('pdb_xchem_matches.txt', 'w') as f:
    for pdbid in unique_matches:
             f.write(f'{pdbid},')

854


In [ ]:
xcdb_f = open('smiles_list.txt', 'r').read()
xcdb_smiles = xcdb_f.split(',')

In [ ]:
can_mol = Chem.MolFromSmiles(new_smiles,isomericSmiles=False)

In [ ]:
can_mol = Chem.MolFromSmiles('CCCC')

In [ ]:
Chem.MolToSmiles(can_mol, isomericSmiles=False)

In [ ]:
def xchem_pdb_match(row, xcdb_fps):
    if row['compound_id']:
        try:
            cmpd_ids = row['compound_id'].split(',')
        except:
            continue
    for cid in cmpd_ids:
        if cid in non_ligs: 
            continue
        try:
            chem_desc = pypdb.describe_chemical(f"{cid}")
            new_smiles = chem_desc["describeHet"]["ligandInfo"]["ligand"]["smiles"]
            can_mol = Chem.MolFromSmiles(new_smiles)
            can_no_h = Chem.rdmolops.RemoveHs(can_mol, sanitize=True)
            can_smi = Chem.MolToSmiles(can_no_h, isomericSmiles=False)
            can_can_mol = Chem.MolFromSmiles(can_smi)
            can_fp = AllChem.GetMorganFingerprint(can_can_mol,2)
                sim = DataStructs.DiceSimilarity(can_fp,comparison)
                if sim > 0.9:
                    print(sim)
                    print(can_smi)